In [1]:
import glob
import os
import re

In [2]:
authors_folder = '../../content/authors/'
author_mds = glob.glob(os.path.join(authors_folder,"*","_index.md"))

author_dict = {"first": {}, "last": {}}
name_re = re.compile("title: (.+?)\n")

In [3]:
for author_md in author_mds:
    author = os.path.basename(os.path.dirname(author_md))
    with open(author_md,'r') as f:
        markdown = f.read()
    if not 'superuser:' in markdown:
        phrase = '# Is this the primary user of the site?\n'
        ind = markdown.find(phrase) + len(phrase)
        markdown = markdown[:ind] + 'superuser: false\n' + markdown[ind:]
    if not 'authors:' in markdown:
        markdown = markdown.split('\n')
        markdown.insert(4,f"# Username (this should match the folder name)\nauthors:\n  - {author}\n")
        markdown = '\n'.join(markdown)
    with open(author_md,'w') as f:
        f.write(markdown)
    names = name_re.search(markdown).group(1).split()
    first_name = names[0]
    last_name = names[-1]
    author_dict['first'][first_name] = author
    author_dict['last'][last_name] = author

In [4]:
author_dict['first']['Heather'] = 'admin'
author_dict['first']['H.'] = 'admin'
first_names = author_dict['first']
last_names = author_dict['last']

In [5]:
pub_folder = '../../content/publication/'
pub_mds = glob.glob(os.path.join(pub_folder,"*","index.md"))

In [6]:
for pub_md in pub_mds:
    with open(pub_md,'r') as f:
        markdown = f.readlines()
    in_authors = False
    test = False
    for i, line in enumerate(markdown):
        if 'authors:' in line:
            in_authors = True
            continue
        if not(in_authors):
            continue
        if ":" in line:
            break
        name = line.split()
        last_name = name[-1]
        first_name = name[1]
        if not last_name in last_names:
            continue
        if not first_name in first_names:
            continue

        if first_names[first_name] == last_names[last_name]:
            author = author_dict['first'][first_name]
            markdown[i] = f"- {author}\n"
            test = True
    with open(pub_md,'w') as f:
        f.writelines(markdown)
